In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

In [3]:
sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)

In [4]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions = np.dot(feature_matrix, weights)
    return(predictions)

In [5]:
def normalize_features(features):
    norms = np.linalg.norm(features, axis=0)
    normalized_features = features / norms
    return (normalized_features, norms)

In [6]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print features
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print norms
# should print
# [5.  10.  15.]

[[ 0.6  0.6  0.6]
 [ 0.8  0.8  0.8]]
[  5.  10.  15.]


Review of Coordinate Descent

7. We seek to obtain a sparse set of weights by minimizing the LASSO cost function

SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).

(By convention, we do not include w[0] in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use coordinate descent: at each iteration, we will fix all weights but weight i and find the value of weight i that minimizes the objective. That is, we look for

argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]

where all weights other than w[i] are held to be constant. We will optimize one w[i] at a time, circling through the weights multiple times.

    Pick a coordinate i
    Compute w[i] that minimizes the LASSO cost function
    Repeat the two steps for all coordinates, multiple times
    
       ┌ (ro[i] + lambda/2)    if ro[i] < -lambda/2
w[i] = ├ 0                     if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)    if ro[i] > lambda/2
       
where

ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].

In [7]:
df_train = pd.read_csv('kc_house_train_data.csv', dtype=dtype_dict)
df_test = pd.read_csv('kc_house_test_data.csv', dtype=dtype_dict)

In [8]:
print df_train.shape, df_test.shape

(17384, 21) (4229, 21)


In [9]:
sales['constant'] = 1

In [10]:
simple_features = sales[['constant','sqft_living','bedrooms']].values
simple_output = sales['price'].values

In [11]:
weights = np.array([1., 4., 1.])

In [34]:
simple_feature_matrix, norms = normalize_features(simple_features)

In [35]:
simple_features, simple_feature_matrix, norms

(array([[  1.00000000e+00,   1.18000000e+03,   3.00000000e+00],
        [  1.00000000e+00,   2.57000000e+03,   3.00000000e+00],
        [  1.00000000e+00,   7.70000000e+02,   2.00000000e+00],
        ..., 
        [  1.00000000e+00,   1.02000000e+03,   2.00000000e+00],
        [  1.00000000e+00,   1.60000000e+03,   3.00000000e+00],
        [  1.00000000e+00,   1.02000000e+03,   2.00000000e+00]]),
 array([[ 0.00680209,  0.00353021,  0.00583571],
        [ 0.00680209,  0.00768869,  0.00583571],
        [ 0.00680209,  0.00230361,  0.00389048],
        ..., 
        [ 0.00680209,  0.00305154,  0.00389048],
        [ 0.00680209,  0.00478673,  0.00583571],
        [ 0.00680209,  0.00305154,  0.00389048]]),
 array([  1.47013605e+02,   3.34257264e+05,   5.14075870e+02]))

In [14]:
prediction = predict_output(simple_feature_matrix, weights)

In [15]:
prediction.shape, weights

((21613,), array([ 1.,  4.,  1.]))

In [16]:
prediction

array([ 0.02675867,  0.04339256,  0.01990703, ...,  0.02289873,
        0.03178473,  0.02289873])

In [17]:
range(simple_feature_matrix.shape[1])

[0, 1, 2]

In [18]:
ro = np.array([0., 0., 0.])
for i in range(simple_feature_matrix.shape[1]):
    ro[i] = np.dot(simple_feature_matrix[:,i], (simple_output - prediction + (weights[i] * simple_feature_matrix[:,i])))
#    ro[i] = sum(simple_feature_matrix[:,i] * (simple_output - prediction + (weights[i] * simple_feature_matrix[:,i])))
#    ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i])]

In [19]:
ro

array([ 79400300.01452321,  87939470.82325152,  80966698.66623905])

In [20]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix, weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = np.dot(feature_matrix[:,i], (output - (prediction - (weights[i] * feature_matrix[:,i]))))
    
    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2
    else:
        new_weight_i = 0.
    
    return new_weight_i

In [21]:
# should print 0.425558846691
import math
print lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],
                   [2./math.sqrt(13),3./math.sqrt(10)]]), np.array([1., 1.]), np.array([1., 4.]), 0.1)

0.425558846691


In [27]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    converged = False 
    weights = np.array(initial_weights) # make sure it's a numpy array
    coordinate_change = np.zeros(len(weights))
    
#    for k in range(1,100):
    while not converged:
        for i in range(len(weights)):
            old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
        # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
        #     and old values for weight[i], ..., weight[d-1]
            weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
        # use old_weights_i to compute change in coordinate
            coordinate_change[i] = np.abs(old_weights_i - weights[i])
#        print coordinate_change, max(coordinate_change)
        if max(coordinate_change) < tolerance:
            converged = True
    
    return weights

In [23]:
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0
simple_feature_matrix, simple_output

(array([[ 0.00680209,  0.00353021,  0.00583571],
        [ 0.00680209,  0.00768869,  0.00583571],
        [ 0.00680209,  0.00230361,  0.00389048],
        ..., 
        [ 0.00680209,  0.00305154,  0.00389048],
        [ 0.00680209,  0.00478673,  0.00583571],
        [ 0.00680209,  0.00305154,  0.00389048]]),
 array([ 221900.,  538000.,  180000., ...,  402101.,  400000.,  325000.]))

In [24]:
lasso_weights = lasso_cyclical_coordinate_descent(simple_feature_matrix, simple_output, initial_weights, l1_penalty, tolerance)

[ 79400304.63764513  10305258.70494878    299724.16960754] 79400304.6376
[ 9138168.37642823  8642337.05981864   299724.16960754] 9138168.37643
[ 8194809.51838306  7213612.49873429        0.        ] 8194809.51838
[ 6598905.08191977  6036579.90886382        0.        ] 6598905.08192
[ 5522173.23081964  5051601.67703672        0.        ] 5522173.23082
[ 4621129.84087856  4227340.62808831        0.        ] 4621129.84088
[ 3867108.13182595  3537572.81915549        0.        ] 3867108.13183
[ 3236118.83201169  2960353.22246729        0.        ] 3236118.83201
[ 2708086.95746362  2477317.54221924        0.        ] 2708086.95746
[ 2266213.12438758  2073097.9527741         0.        ] 2266213.12439
[ 1896439.07519021  1734834.16984525        0.        ] 1896439.07519
[ 1587000.41368857  1451764.29933538        0.        ] 1587000.41369
[ 1328052.31973784  1214882.44666699        0.        ] 1328052.31974
[ 1111356.33535265  1016652.19340119        0.        ] 1111356.33535
[ 930018.25739287

In [25]:
lasso_weights

array([ 21624997.95951869,  63157247.2078898 ,         0.        ])

In [26]:
print sum((predict_output(simple_feature_matrix, lasso_weights) - simple_output)**2)

1.63049247672e+15


In [45]:
all_features = ['constant',
                'bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

In [46]:
df_train['constant'] = 1
df_test['constant'] = 1

In [47]:
X_train, norms = normalize_features(df_train[all_features].values)

In [48]:
X_train.shape, norms

((17384, 14), array([  1.31848398e+02,   4.60040216e+02,   2.96850552e+02,
          2.99962419e+05,   5.81709718e+06,   2.09458827e+02,
          1.15325626e+01,   1.05933942e+02,   4.57793622e+02,
          1.02101959e+03,   2.59726472e+05,   7.01224951e+04,
          2.59922094e+05,   5.36953839e+04]))

In [49]:
y_train = df_train['price'].values

In [50]:
initial_weights = np.zeros(len(all_features))
l1_penalty = 1e7
tolerance = 1.0
weight1e7 = lasso_cyclical_coordinate_descent(X_train, y_train, initial_weights, l1_penalty, tolerance)

In [52]:
print(zip(all_features, weight1e7))

[('constant', 24429600.234403357), ('bedrooms', 0.0), ('bathrooms', 0.0), ('sqft_living', 48389174.771548554), ('sqft_lot', 0.0), ('floors', 0.0), ('waterfront', 3317511.2149216477), ('view', 7329961.8117143344), ('condition', 0.0), ('grade', 0.0), ('sqft_above', 0.0), ('sqft_basement', 0.0), ('yr_built', 0.0), ('yr_renovated', 0.0)]


In [53]:
l1_penalty = 1e8
weight1e8 = lasso_cyclical_coordinate_descent(X_train, y_train, initial_weights, l1_penalty, tolerance)

In [54]:
print(zip(all_features, weight1e8))

[('constant', 71114625.714887127), ('bedrooms', 0.0), ('bathrooms', 0.0), ('sqft_living', 0.0), ('sqft_lot', 0.0), ('floors', 0.0), ('waterfront', 0.0), ('view', 0.0), ('condition', 0.0), ('grade', 0.0), ('sqft_above', 0.0), ('sqft_basement', 0.0), ('yr_built', 0.0), ('yr_renovated', 0.0)]


In [55]:
l1_penalty = 1e4
tolerance = 5e5
weight1e4 = lasso_cyclical_coordinate_descent(X_train, y_train, initial_weights, l1_penalty, tolerance)

In [56]:
print(zip(all_features, weight1e4))

[('constant', 78564738.341568455), ('bedrooms', -22097398.924305033), ('bathrooms', 12791071.872785015), ('sqft_living', 93808088.092812285), ('sqft_lot', -2013172.7570497463), ('floors', -4219184.9326500716), ('waterfront', 6482842.8175350325), ('view', 7127408.5348068327), ('condition', 5001664.8546971167), ('grade', 14327518.437141208), ('sqft_above', -15770959.152374217), ('sqft_basement', -5159591.2221315317), ('yr_built', -84495341.768439025), ('yr_renovated', 2824439.4970368915)]


In [59]:
normalized_weight1e7 = weight1e7 / norms
normalized_weight1e8 = weight1e8 / norms
normalized_weight1e4 = weight1e4 / norms

In [60]:
normalized_weight1e7[3]

161.31745764611625

In [61]:
X_test = df_test[all_features].values
y_test = df_test['price'].values
predict1e7 = predict_output(X_test, normalized_weight1e7)
predict1e8 = predict_output(X_test, normalized_weight1e8)
predict1e4 = predict_output(X_test, normalized_weight1e4)

In [63]:
print "RSS weight 1e7: " + str(sum((predict1e7 - y_test)**2))
print "RSS weight 1e8: " + str(sum((predict1e8 - y_test)**2))
print "RSS weight 1e4: " + str(sum((predict1e4 - y_test)**2))

RSS weight 1e7: 2.7596207592e+14
RSS weight 1e8: 5.37166151497e+14
RSS weight 1e4: 2.28459958971e+14
